In [ ]:
# Copyright 2026 50Hertz Transmission GmbH and Elia Transmission Belgium
#
# This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0.
# If a copy of the MPL was not distributed with this file,
# you can obtain one at https://mozilla.org/MPL/2.0/.
# Mozilla Public License, version 2.0

Pandapower

In [ ]:
from pandapower.converter import from_cim
from toop_engine_grid_helpers.pandapower.pandapower_id_helpers  import get_globally_unique_id 
from toop_engine_interfaces.nminus1_definition import Nminus1Definition, GridElement, Contingency
from toop_engine_contingency_analysis.pandapower import run_contingency_analysis_pandapower


net = from_cim.from_cim("path_to_cim.zip")
monitored_lines = [get_globally_unique_id(line_id, "line") for line_id in net.line.index]
monitored_trafos = [get_globally_unique_id(trafo_id, "trafo") for trafo_id in net.trafo.index]
monitored_trafo3ws = [get_globally_unique_id(trafo_id, "trafo3w") for trafo_id in net.trafo3w.index]
outaged_lines = monitored_lines + monitored_trafos
monitored_buses = [get_globally_unique_id(bus_id, "bus") for bus_id in net.bus.index]

nminus1_def = Nminus1Definition(
    monitored_elements=[GridElement(id=line_id, type="line", kind="branch") for line_id in monitored_lines] + [GridElement(id=trafo_id, type="trafo", kind="branch") for trafo_id in monitored_trafos] + [GridElement(id=trafo_id, type="trafo3w", kind="branch") for trafo_id in monitored_trafo3ws] + [GridElement(id=bus_id, type="bus", kind="bus") for bus_id in monitored_buses],
    contingencies=[Contingency(id=line_id, elements=[GridElement(id=line_id, type="line", kind="branch")]) for line_id in outaged_lines],
    id_type="unique_pandapower"
    )

In [ ]:
lf_results_pandapower = run_contingency_analysis_pandapower(net, nminus1_def, job_id="test", timestep=0, method="ac" n_processes=15)


In [ ]:
n_monitored = len(monitored_lines)*2 + len(monitored_trafos)*2 + len(monitored_trafo3ws)*3
n_outaged = len(outaged_lines)
expected = n_monitored * n_outaged
expected

In [ ]:
lf_results_pandapower.branch_results

Powsybl

In [ ]:
from toop_engine_interfaces.nminus1_definition import Nminus1Definition, GridElement, Contingency
import pypowsybl
from toop_engine_contingency_analysis.ac_loadflow_service import get_ac_loadflow_results
from toop_engine_contingency_analysis.pypowsybl import run_contingency_analysis_powsybl
cgmes_file = "path/to/cgmes.zip" # Could also be ucte or anything else
net = pypowsybl.network.load(cgmes_file)
lines = net.get_lines(attributes=[])
trafos = net.get_2_windings_transformers(attributes=[])
trafo_3ws = net.get_3_windings_transformers(attributes=[])
nodes = net.get_buses(attributes=[])
monitored_lines = [line_id for line_id in lines.index]
monitored_trafos = [trafo_id for trafo_id in trafos.index]
monitored_trafo3ws = [trafo_id for trafo_id in trafo_3ws.index]
outaged_lines = monitored_lines + monitored_trafos
monitored_buses = [bus_id for bus_id in nodes.index]

pow_nminus1_def = Nminus1Definition(
    monitored_elements=[GridElement(id=line_id, type="line", kind="branch") for line_id in monitored_lines] + [GridElement(id=trafo_id, type="trafo", kind="branch") for trafo_id in monitored_trafos] + [GridElement(id=trafo_id, type="trafo3w", kind="branch") for trafo_id in monitored_trafo3ws] + [GridElement(id=bus_id, type="bus", kind="bus") for bus_id in monitored_buses],
    contingencies=[Contingency(id=line_id, elements=[GridElement(id=line_id, type="line", kind="branch")]) for line_id in outaged_lines]
    )

lf_results = run_contingency_analysis_powsybl(net, pow_nminus1_def, "test_job", timestep=0, method="ac", n_processes=15)


In [ ]:
n_monitored = len(monitored_lines)*2 + len(monitored_trafos)*2 + len(monitored_trafo3ws)*3
n_outaged = len(outaged_lines)
expected = n_monitored * n_outaged
expected

In [ ]:
lf_results_powsybl = get_ac_loadflow_results(net, pow_nminus1_def, job_id="test", n_processes=15)

In [ ]:
import pandera as pa
config = pa.config.PanderaConfig(validation_enabled=False, validation_depth=None)
pa.config.reset_config_context(config)
lf_results_powsybl = get_ac_loadflow_results(net, pow_nminus1_def, job_id="test", n_processes=15)

In [ ]:
%%prun
get_ac_loadflow_results(net, pow_nminus1_def, job_id="test", n_processes=15)